In [11]:
import pandas as pd
import numpy as np
import pickle
from src.ml.competence_clusterization.preprocessing.initial_preproc import *
from sklearn.metrics import silhouette_samples, silhouette_score

In [2]:
df_full = pd.read_csv('X:/dataset.csv', delimiter='|', low_memory=False)

In [3]:
df_full.shape

(23213883, 17)

In [4]:
ds_skills = df_full[(df_full["skill"].notna())]

In [5]:
ds_skills.shape

(14858517, 17)

In [6]:
ds_skills.head()

,parsing_date,resource_title,vacancy_id,publication_date,description,company_name,exp_min,exp_max,lower_threshold,upper_threshold,currency,gross,vacancy_title,skill_id,skill,responsibility_id,responsibility
2,2023-02-19 00:00:00,hh,2553058,2013-04-11 16:46:51,Компания ;Сбербанк-Технологии; приглашает Ру...,SberTech,1,3,23000.0,45000.0,NaN,NaN,Руководитель проектов (Тарифные планы),10132423.0,Опыт работы с 1С:8.2,NaN,NaN
3,2023-02-19 00:00:00,hh,2553058,2013-04-11 16:46:51,Компания ;Сбербанк-Технологии; приглашает Ру...,SberTech,1,3,23000.0,45000.0,NaN,NaN,Руководитель проектов (Тарифные планы),1841.0,Аналитический склад ума,NaN,NaN
4,2023-02-19 00:00:00,hh,2553058,2013-04-11 16:46:51,Компания ;Сбербанк-Технологии; приглашает Ру...,SberTech,1,3,23000.0,45000.0,NaN,NaN,Руководитель проектов (Тарифные планы),10132421.0,компании (используются типовые конфигурации УТ...,NaN,NaN
5,2023-02-19 00:00:00,hh,2553058,2013-04-11 16:46:51,Компания ;Сбербанк-Технологии; приглашает Ру...,SberTech,1,3,23000.0,45000.0,NaN,NaN,Руководитель проектов (Тарифные планы),10132420.0,Требования: Высшее или неполное высшее образ...,NaN,NaN
6,2023-02-19 00:00:00,hh,2553058,2013-04-11 16:46:51,Компания ;Сбербанк-Технологии; приглашает Ру...,SberTech,1,3,23000.0,45000.0,NaN,NaN,Руководитель проектов (Тарифные планы),2325808.0,Участие в разработке инструкций и регламентов ...,NaN,NaN


In [7]:
ds_skills['preprocessed_data'] = ds_skills['skill'].apply(lowercase)\
                                                   .apply(remove_digits)\
                                                   .apply(remove_punctuation)\
                                                   .apply(remove_whitespaces)\
                                                   .apply(remove_stop_words)\
                                                   .apply(remove_alone_chars)

C:\Users\Dmitry\AppData\Local\Temp\ipykernel_2932\3884889856.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds_skills['preprocessed_data'] = ds_skills['skill'].apply(lowercase)\


In [8]:
string_to_stem = "||".join(ds_skills['preprocessed_data'][:7000000])
stemmed_data = stem(string_to_stem)

C:\Users\Dmitry\AppData\Local\Temp\ipykernel_2932\3027119278.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  string_to_stem = "||".join(ds_skills['preprocessed_data'][:7000000])


In [9]:
string_to_stem2 = "||".join(ds_skills['preprocessed_data'][7000000:])
stemmed_data2 = stem(string_to_stem2)

C:\Users\Dmitry\AppData\Local\Temp\ipykernel_2932\897987848.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  string_to_stem2 = "||".join(ds_skills['preprocessed_data'][7000000:])


In [10]:
string_all = stemmed_data + "||" + stemmed_data2


In [11]:
ds_skills['stemmed_data'] = string_all.split('||')

C:\Users\Dmitry\AppData\Local\Temp\ipykernel_2932\3033889215.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds_skills['stemmed_data'] = string_all.split('||')


In [12]:
ds_skills.to_csv('X:/dataset_with_append.csv', sep='|')

In [2]:
df = pd.read_csv('X:/dataset_with_append.csv', delimiter='|', low_memory=False)

In [14]:
with open('data_by_line.txt', 'w', encoding='utf-8') as f:
    for line in df.stemmed_data.values:
        f.write(str(line) + '\n')

In [2]:
data = []
with open('data_by_line.txt', 'r',encoding='utf-8') as f:
    for line in f:
        data.append(line[:-1].split())

In [5]:
print(data[:10])

[['опыт', 'работа', 'DсDD'], ['аналитический', 'склад', 'ум'], ['компания', 'использоваться', 'типовой', 'конфигурация', 'ута', 'бух'], ['требование', 'высокий', 'неполный', 'высокий', 'образование', 'желательно', 'экономический'], ['участие', 'разработка', 'инструкция', 'регламент', 'работа', 'программа'], ['участие', 'составление', 'тз', 'планаграфик', 'работа', 'конфигурированиюDс'], ['мужчина', 'высокий', 'образование'], ['опыт', 'планирование', 'управление', 'контекстный', 'рекламный', 'компания', 'служба', 'yandex', 'direct', 'google', 'adwords', 'begun', 'опыт', 'работа', 'google', 'analytics'], ['владение', 'inet', 'технология'], ['грамотность']]


In [3]:
from gensim.models import FastText
from gensim.test.utils import datapath, common_texts

#model_corpus_file = datapath('data_by_line.txt')
#data = df.stemmed_data.str.split().to_list()


In [6]:
model = FastText(sentences=data, vector_size=35, window=3, min_count=1, epochs=10)
model.save('FastText_model_10epochs_35vs')

In [4]:
model = FastText.load('FastText_model_10epochs_35vs')


In [5]:
documents = []
for document in data:
    word_vectors = []
    if document:
        for word in document: # or your logic for separating tokens
            word_vectors.append(model.wv[word])
        documents.append(word_vectors)
print(documents[:10])
# resulting in an n * 120 -- that is, `Word2Vec:size`-- array
#document_matrix = np.concatenate(documents)

[[array([ 7.1409926 ,  2.0438483 ,  3.0850518 , -0.79413867,  7.2658024 ,
        2.7019577 ,  4.588082  , -0.8336994 , -0.24877119,  7.268665  ,
       -4.820719  , -3.7394783 , -0.15347238, -6.82093   , -3.612172  ,
       -4.1876316 , -1.3660282 , -3.3291266 ,  4.2283664 ,  2.524864  ,
        3.1225798 ,  2.8575866 , -3.4781318 , -6.1971774 ,  4.202084  ,
        1.6683589 ,  2.4339607 , -2.5799806 , -2.8824563 , -4.300989  ,
        4.9194183 , -3.1806314 , -3.1758666 ,  1.7153634 , -0.4071429 ],
      dtype=float32), array([-2.641918  ,  1.3036511 ,  2.4598827 ,  1.5937452 , -2.2755284 ,
        2.1990845 ,  3.489985  , -5.6332493 , -3.082744  ,  4.279767  ,
        2.8858185 ,  2.0252664 , -3.6162615 ,  0.8313726 ,  1.1736288 ,
        1.4213375 ,  1.8187186 ,  1.535529  , -2.128717  , -2.6889849 ,
        8.421101  , -2.7903545 ,  2.2462087 , -3.942202  , -1.2980433 ,
        0.7533353 , -1.335578  ,  0.48170617,  0.81690025, -5.5830946 ,
        1.7514417 ,  2.7403133 , -3.460

In [23]:
del model
del documents
del data

NameError: name 'model' is not defined

In [12]:
print(len(documents))

14858517


In [15]:
from tqdm import tqdm
vector_len = 35
concat_list = []
for doc in tqdm(documents):
    new_arr = [0 for _ in range(vector_len)]
    for word in doc:
        for ind, num in enumerate(word):
            new_arr[ind] += num
    concat_list.append(new_arr)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14858517/14858517 [15:19<00:00, 16151.04it/s]


In [20]:
print(len(concat_list))
numpy_X = np.array(concat_list)

14858517


In [24]:
with open('concat_list_35.npy', 'wb') as f:
    np.save(f, numpy_X)

In [7]:
with open('concat_list_35.npy', 'rb') as f:
    numpy_X = np.load(f)

In [4]:
def tsne_plot(model):
    labels = []
    tokens = []
    words = model.wv.index_to_key[100:500]
    for word in model.wv.index_to_key[100:500]: # not using the first 20 as they are mostly most common words
#     for word in words:
        tokens.append(model.wv.get_vector(word))
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(np.asarray(tokens))

    x = []
    y = []
    labeling = []
#     for value in new_values:
#         x.append(value[0])
#         y.append(value[1])
    for index, value in enumerate(new_values):
        if labels[index] in words:
            x.append(value[0])
            y.append(value[1])
            labeling.append(labels[index])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
#         plt.annotate(labels[i],
#                      xy=(x[i], y[i]),
#                      xytext=(5, 2),
#                      textcoords='offset points',
#                      ha='right',
#                      va='bottom')
        plt.annotate(labeling[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

In [3]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np

tsne_plot(model)
#print(model.wv.index_to_key[:100])

NameError: name 'tsne_plot' is not defined

In [ ]:
from sklearn.cluster import DBSCAN, OPTICS
import pickle

dbscan = OPTICS(cluster_method='dbscan', n_jobs=-1, eps=0.2, leaf_size=20, algorithm='ball_tree').fit(numpy_X)
#dbscan = DBSCAN(min_samples=20, eps=0.1).fit(numpy_X)
filename = 'X:/dbscan_model.sav'
pickle.dump(dbscan, open(filename, 'wb'))

In [3]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=60, init='k-means++', n_init=10)
kmeans.fit(numpy_X)

KMeans(n_clusters=60, n_init=10)

In [4]:

filename = 'X:/kmeans_model.sav'
pickle.dump(kmeans, open(filename, 'wb'))

In [5]:
with open("X:/kmeans_model.sav", "rb") as f:
    kmeans = pickle.load(f)

In [ ]:
cluster_labels = kmeans.predict(numpy_X)
silhouette_avg = silhouette_score(numpy_X, cluster_labels)
print('Silhoette score is: ' + silhouette_avg)